In [1]:
from dotenv import load_dotenv
import os, requests, sys, json
from IPython.display import JSON
from pathlib import Path
import logging.handlers
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy.signal import convolve2d
import warnings
import seaborn as sns

from os import listdir
from os.path import isfile, join
warnings.filterwarnings("ignore")

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from Helpers.Exception_Handling import Exception_Handling as eh
from Helpers.Mongo_Interface import Mongo_Interface as mi

env_path = Path(fr"{os.environ['USERPROFILE']}\.env") 
load_dotenv(dotenv_path=env_path)

True

In [2]:
path = r'../data/archive'
onlyfiles = os.listdir(path)

In [4]:
print(len(onlyfiles))
onlyfiles[0:5]

1855180


['10000297.json',
 '10000297_info.json',
 '10000709.json',
 '10000709_info.json',
 '10000716.json']

In [5]:
onlyfiles = list(filter(lambda x: 'info' not in x, onlyfiles))

In [6]:
print(len(onlyfiles))
onlyfiles[0:5]

927590


['10000297.json',
 '10000709.json',
 '10000716.json',
 '10001126.json',
 '10001127.json']

In [7]:
import threading

In [8]:
thread_manager = {}
thread_limit = 100
results = []

import time
def second_thread(file_name):
    global results
    document = json.load(open(f"{path}\\{file_name}"))
    results.append(document)

for i in range(thread_limit):
    thread_manager[i] = None

for file_name in tqdm(onlyfiles):
    while True:
        thread_started = False
        for i in range(thread_limit):
            if thread_manager[i] == None or not thread_manager[i].is_alive():
                thread_manager[i] = threading.Thread(target=second_thread, args=(file_name,))
                thread_manager[i].start()
                thread_started = True
                break
        if thread_started:
            break
        
len(results)

 58%|█████████████████████████████████████████▉                              | 540667/927590 [06:09<03:20, 1929.57it/s]Exception in thread Thread-540804 (second_thread):
Traceback (most recent call last):
  File "C:\Users\ryans\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\ryans\AppData\Local\Programs\Python\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ryans\AppData\Local\Temp\ipykernel_30644\320621707.py", line 8, in second_thread
  File "C:\Users\ryans\AppData\Local\Programs\Python\Python310\lib\json\__init__.py", line 293, in load
    return loads(fp.read(),
  File "C:\Users\ryans\AppData\Local\Programs\Python\Python310\lib\json\__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "C:\Users\ryans\AppData\Local\Programs\Python\Python310\lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 

927589

In [64]:
lol = os.path.dirname(os.path.abspath("__file__"))
target = os.path.join(lol, 'data\\working\\documents.json')
target

'D:\\source\\repos\\KaggleConnectX\\notebooks\\data\\working\\documents.json'

In [9]:


with open(r"D:\source\repos\KaggleConnectX\data\working\documents.json", "w+") as outfile:
    outfile.write(json.dumps({'documents':results}))

KeyboardInterrupt: 

In [ ]:
doc = {}
with open(r"N:\documents.json", "r") as outfile:
    doc = json.load(outfile)
doc['documents'][0]

In [9]:
horizontal_kernel = np.array([[ 1, 1, 1, 1]])
vertical_kernel = np.transpose(horizontal_kernel)
diag1_kernel = np.eye(4, dtype=np.uint8)
diag2_kernel = np.fliplr(diag1_kernel)
detection_kernels = [horizontal_kernel, vertical_kernel, diag1_kernel, diag2_kernel]

def winning_move(board, player):
    for kernel in detection_kernels:
        if (convolve2d(board == player, kernel, mode="valid") == 4).any():
            return True
    return False

In [10]:
columns = [item for item in range(1, 43)]
columns.append('board_series')
columns.append('winner')
columns.append('looser')
columns.append('is_stale')


df = pd.DataFrame(columns=columns)
obj =''
rows = []

for i in tqdm(range(len(results))):
    flat_board = results[i]['steps'][-1:][0][0]['observation']['board'].copy()
    if len(results[i]['steps']) < 3:
           continue
    board = np.resize(flat_board, (6,7))
    flat_board.append(results[i]['steps'])
    if winning_move(board, 1):
        flat_board.append(1)
        flat_board.append(2)
        flat_board.append(False)
    elif winning_move(board, 2):
        flat_board.append(2)
        flat_board.append(1)
        flat_board.append(False)
    else:
        flat_board.append(None)
        flat_board.append(None)
        flat_board.append(True)
    rows.append(flat_board)
    
df2 = pd.DataFrame(data=rows, columns=columns)
df = pd.concat([df, df2], ignore_index=True)
df

100%|███████████████████████████████████████████████████████████████████████| 927589/927589 [01:03<00:00, 14568.60it/s]


,1,2,3,4,5,6,7,8,9,10,...,37,38,39,40,41,42,board_series,winner,looser,is_stale
0,2,2,0,2,1,1,2,1,1,0,...,2,1,1,1,2,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",2.0,1.0,False
1,1,0,2,2,2,1,0,2,0,2,...,2,2,1,1,1,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False
2,2,1,0,2,2,0,1,1,2,0,...,2,1,1,2,1,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False
3,0,2,2,2,1,0,0,0,1,1,...,2,1,1,2,1,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",2.0,1.0,False
4,1,0,2,2,0,1,1,2,0,2,...,1,2,1,2,1,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
925236,1,2,2,2,0,1,2,2,1,1,...,2,1,1,1,2,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False
925237,2,1,0,2,2,1,0,2,2,1,...,1,1,1,2,2,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False
925238,1,0,0,1,0,0,0,1,1,0,...,2,2,1,2,2,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",2.0,1.0,False
925239,2,1,1,1,0,1,0,2,2,1,...,2,1,1,1,2,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False


In [12]:
df[df['winner'] == 1]

,1,2,3,4,5,6,7,8,9,10,...,37,38,39,40,41,42,board_series,winner,looser,is_stale
0,2,1,0,2,2,0,1,1,2,0,...,2,1,1,2,1,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False
2,1,1,2,2,0,0,0,2,2,1,...,1,1,1,2,2,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False
3,1,0,2,2,2,1,0,2,0,2,...,2,2,1,1,1,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False
6,1,0,2,2,0,1,1,2,0,2,...,1,2,1,2,1,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False
8,2,0,1,1,0,1,2,2,0,2,...,2,2,1,2,2,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416352,0,1,0,1,0,0,0,0,2,0,...,1,1,1,2,1,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False
416354,1,2,2,2,0,1,2,2,1,1,...,2,1,1,1,2,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False
416355,2,0,2,2,2,1,1,1,0,1,...,1,1,1,2,2,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False
416357,2,1,1,1,0,1,0,2,2,1,...,2,1,1,1,2,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False


In [8]:
#df.loc[2, 'board_series']

In [11]:
columns2 = [item for item in range(1, 43)]
columns2.append('target')
columns2.append('winner')

df_actions = pd.DataFrame(columns=columns2)
working_set = df[df['is_stale'] == False]

preload = []

for i in tqdm(working_set.index):
    series = working_set.loc[i, 'board_series'].copy()
    winning_player = working_set.loc[i, 'winner']
    if len(series) > 3:
        if winning_player == 1:
            for j in range(2, len(series), 2):
                if len(series) > j + 1:
                    flat_board = working_set.loc[i, 'board_series'][j][0]['observation']['board'].copy()
                    #print(len(flat_board))
                    seen = np.resize(working_set.loc[i, 'board_series'][j][0]['observation']['board'], (6,7))
                    action = np.resize(working_set.loc[i, 'board_series'][j + 1][0]['observation']['board'], (6,7))
                    result = action - seen
                    target = np.unravel_index(result.argmax(), result.shape)[1]
                    flat_board.append(target)
                    flat_board.append(winning_player)
                    #print(flat_board)
                    preload.append(flat_board)
                    #preload.append(winning_player)
                    #df2 = pd.DataFrame(data=[flat_board], columns=columns2)
        elif winning_player == 2:
            for j in range(1, len(series), 2):
                if len(series) > j + 1:
                    flat_board = working_set.loc[i, 'board_series'][j][0]['observation']['board'].copy()
                    #print(len(flat_board))
                    seen = np.resize(working_set.loc[i, 'board_series'][j][0]['observation']['board'], (6,7))
                    action = np.resize(working_set.loc[i, 'board_series'][j + 1][0]['observation']['board'], (6,7))
                    result = action - seen
                    target = np.unravel_index(result.argmax(), result.shape)[1]
                    flat_board.append(target)
                    flat_board.append(winning_player)
                    #print(flat_board)
                    preload.append(flat_board)
                    #preload.append(winning_player)
                    #df2 = pd.DataFrame(data=[flat_board], columns=columns2)

df_actions = pd.DataFrame(data=preload, columns=columns2)

df_actions

100%|████████████████████████████████████████████████████████████████████████| 886777/886777 [11:15<00:00, 1313.25it/s]


,1,2,3,4,5,6,7,8,9,10,...,35,36,37,38,39,40,41,42,target,winner
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,3,2.0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,3,2.0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,3,2.0
3,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,5,2.0
4,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,1,1,2,0,4,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14452142,0,0,0,1,0,1,0,2,0,0,...,1,2,0,2,1,2,2,2,2,2.0
14452143,0,0,0,1,0,1,0,2,0,2,...,1,2,0,2,1,2,2,2,4,2.0
14452144,0,0,0,1,0,1,0,2,0,2,...,1,2,0,2,1,2,2,2,2,2.0
14452145,0,0,2,1,0,1,0,2,0,2,...,1,2,0,2,1,2,2,2,4,2.0


In [36]:
#columns5 = [item for item in range(1, 43)]
#columns5.append('winner')
#temp = df_actions.copy()
#temp.loc[:,columns5] = temp.loc[:,columns5].replace(1, 50).replace(2,1).replace(50,2)
#temp
#df_actions = pd.concat([df_actions, temp])

In [12]:
df_actions.to_csv(r'D:\source\repos\KaggleConnectX\data\working\full_actions.csv')

In [15]:
df_actions['winner'].unique()

array([1., 2.])

In [14]:
df_actions['tick'] = 1

In [15]:
columns2 = [item for item in range(1, 43)]
columns2.append('target')
columns2.append('winner')
df_actions_set_one = df_actions.groupby(columns2)['tick'].sum().sort_values(ascending=False).reset_index()
df_actions_set_one['tick_layer_two'] = 1
df_actions_set_one
#df_actions_set_one.groupby(columns2)['tick_layer_two'].sum().sort_values(ascending=False)

,1,2,3,4,5,6,7,8,9,10,...,37,38,39,40,41,42,target,winner,tick,tick_layer_two
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,3,1.0,369292,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,3,1.0,296941,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,2,0,0,1,1.0,166250,1
3,0,0,0,0,0,0,0,0,0,0,...,0,2,1,0,0,0,5,1.0,89099,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,2,3,1.0,81618,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1580240,0,0,1,1,2,0,0,0,0,1,...,2,1,1,2,1,2,5,2.0,1,1
1580241,0,0,1,1,2,0,0,0,0,1,...,2,1,1,1,2,1,5,2.0,1,1
1580242,0,0,1,1,2,0,0,0,0,1,...,2,1,1,2,1,1,1,2.0,1,1
1580243,0,0,1,1,2,0,0,0,0,1,...,2,2,1,1,2,2,5,1.0,1,1


In [16]:
df_actions_set_one[0:200]

,1,2,3,4,5,6,7,8,9,10,...,37,38,39,40,41,42,target,winner,tick,tick_layer_two
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,3,1.0,369292,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,3,1.0,296941,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,2,0,0,1,1.0,166250,1
3,0,0,0,0,0,0,0,0,0,0,...,0,2,1,0,0,0,5,1.0,89099,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,2,3,1.0,81618,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,0,0,0,0,0,0,0,0,0,...,2,0,1,1,2,0,1,1.0,9633,1
196,0,0,0,0,0,0,0,0,0,0,...,1,0,1,2,2,0,4,1.0,9485,1
197,0,0,0,0,0,0,0,0,0,0,...,2,2,1,0,1,2,1,1.0,9473,1
198,0,0,0,0,0,0,0,0,0,0,...,2,2,1,0,1,2,3,1.0,9367,1


In [17]:
df_actions_set_two = df_actions_set_one.reset_index()
df_actions_set_two = df_actions_set_two.drop(columns=['index'])

columns3 = [item for item in range(1, 43)]
columns3.append('winner')

df_actions_set_two = df_actions_set_two.drop_duplicates(subset=columns3, keep="first").reset_index()
df_actions_set_two = df_actions_set_two.drop(columns=['index','tick', 'tick_layer_two'])
df_actions_set_two

,1,2,3,4,5,6,7,8,9,10,...,35,36,37,38,39,40,41,42,target,winner
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,3,1.0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,3,1.0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,2,0,0,1,1.0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,1,0,0,0,5,1.0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2,3,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455736,0,0,1,1,2,0,0,0,0,1,...,2,1,2,1,1,2,1,2,5,2.0
1455737,0,0,1,1,2,0,0,0,0,1,...,0,2,2,1,1,1,2,1,5,2.0
1455738,0,0,1,1,2,0,0,0,0,1,...,2,0,2,1,1,2,1,1,1,2.0
1455739,0,0,1,1,2,0,0,0,0,1,...,0,2,2,2,1,1,2,2,5,1.0


In [18]:
df_actions_set_two.to_csv(r'D:\source\repos\KaggleConnectX\data\working\actions.csv')

In [19]:
df_actions_set_two.to_json(r'D:\source\repos\KaggleConnectX\data\working\actions.json')

In [19]:
sub_template = ''
with open('N:\\connectxactions\\bot_template.txt') as f:
    sub_template = f.read()

cavdata = []
with open('N:\\connectxactions\\actions.csv') as f:
    cavdata = f.read()



In [20]:
from io import StringIO

In [21]:
TESTDATA = StringIO("""col1;col2;col3
    1;4.4;99
    2;4.5;200
    3;4.7;65
    4;3.2;140
    """)

pd.read_csv(TESTDATA, sep=";")

,col1,col2,col3
0,1,4.4,99
1,2,4.5,200
2,3,4.7,65
3,4,3.2,140


In [22]:
TESTDATA = StringIO(cavdata)
columns4 = [item for item in range(1, 43)]
columns4.append('winner')

pd.read_csv(TESTDATA, index_col = columns4)

Unnamed: 0  \
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 winner               
0 0 0 0 0 0 0 0 0 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  2  0  0  0  2                0   
                                                                                    2  0  0  0  0  0  0  1  0  0  0  1                1   
                                                                                    0  0  0  0  0  0  0  1  0  0  0  2                2   
                                                                                                         2  0  0  0  1                3   
                                          1  0  0  0  0  0  0  2  0  0  0  0  0  0  1  0  0  0  0  0  0  2  0  0  0  2                4   
...                                                                                                                                 ...   
                                    1  0  1  1  1  0  0  2  0  2  2  2  0  0  2  0  1  2  1  0  0  1  0  2  2  1  0  2          1873445   
                                                                              1  1  2  2  2  0  1  1  2  2  1  1  0  2          1873446   
                                                                                 0  2  2  2  0  1  1  0  2  1  1  0  2          1873447   
                                                                                          1  0  1  1  2  2  2  1  0  2          1873448   
2 2 2 1 2 2 0 2 1 2  1  2  1  0  1  1  2  1  1  1  0  1  2  1  2  2  1  0  2  2  1  1  2  2  0  1  2  1  2  2  1  1  1          1873449   

                                                                                                                             target  
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 winner          
0 0 0 0 0 0 0 0 0 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  2  0  0  0  2            3  
                                                                                    2  0  0  0  0  0  0  1  0  0  0  1            3  
                                                                                    0  0  0  0  0  0  0  1  0  0  0  2            3  
                                                                                                         2  0  0  0  1            3  
                                          1  0  0  0  0  0  0  2  0  0  0  0  0  0  1  0  0  0  0  0  0  2  0  0  0  2            3  
...                                                                                                                             ...  
                                    1  0  1  1  1  0  0  2  0  2  2  2  0  0  2  0  1  2  1  0  0  1  0  2  2  1  0  2            1  
                                                                              1  1  2  2  2  0  1  1  2  2  1  1  0  2            2  
                                                                                 0  2  2  2  0  1  1  0  2  1  1  0  2            2  
                                                                                          1  0  1  1  2  2  2  1  0  2            3  
2 2 2 1 2 2 0 2 1 2  1  2  1  0  1  1  2  1  1  1  0  1  2  1  2  2  1  0  2  2  1  1  2  2  0  1  2  1  2  2  1  1  1            6  

[1873450 rows x 2 columns]

In [23]:
sub_file = sub_template.replace('<REPLACEMEWITHDATA>',f"""
TESTDATA = StringIO('''{cavdata}''')
columns4 = [item for item in range(1, 43)]
columns4.append('winner')
df_map = pd.read_csv(TESTDATA, index_col = columns4)
""")


In [24]:
with open('N:\\connectxactions\\sub.py', 'w') as f:
    f.write(sub_file)

In [25]:
sub_template

'from kaggle_environments import evaluate, make, utils\nimport pandas as pd\nimport numpy as np\nimport random\nimport requests\nfrom tqdm import tqdm\nfrom scipy.signal import convolve2d\nimport sys\nfrom io import StringIO\n\n#result = requests.get(\'https://www.kaggleusercontent.com/episodes/45908179.json\')\n#sys.stdout.write(\'googleresult\')\n#sys.stdout.write(str(len(result.text)))\n\n<REPLACEMEWITHDATA>\n\n\nhorizontal_kernel = np.array([[ 1, 1, 1, 1]])\nvertical_kernel = np.transpose(horizontal_kernel)\ndiag1_kernel = np.eye(4, dtype=np.uint8)\ndiag2_kernel = np.fliplr(diag1_kernel)\ndetection_kernels = [horizontal_kernel, vertical_kernel, diag1_kernel, diag2_kernel]\n\ndef winning_move(board, player):\n    for kernel in detection_kernels:\n        if (convolve2d(board == player, kernel, mode="valid") == 4).any():\n            return True\n    return False\n\ndef smart_move(board, me, enemey, recurse_count = 0, target_chosen=0):\n    sys.stdout.write(f\'me:{me} enemy:{enemey}\